# customized KoNLPy

https://github.com/lovit/customized_konlpy

In [1]:
import itertools
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from ckonlpy.tag import Twitter
twitter = Twitter()
from collections import Counter
from pandas import read_excel
from lxml import etree

### Gensim
import gensim
from gensim import corpora
# from gensim.models.word2vec import Word2Vec
from gensim.models import FastText
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel

from hanpre import funcs as hp
from hanpre import stopwords as sw

/opt/conda/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
def interested_words():
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch_.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
    keywords = df['핵심단어']
    subkeywords = df['대체어']
    interested_words = df['키워드']
    return keywords, subkeywords, interested_words

my_weights = [
    ('num_nouns', -0.1),
    ('num_words', -0.2),
    ('no_noun', -1),
    ('len_sum_of_nouns', 0.2)
]

def my_evaluate_function(candidate):
    num_nouns = len([word for word, pos, begin, e in candidate if pos == 'Noun'])
    num_words = len(candidate)
    has_no_nouns = (num_nouns == 0)
    len_sum_of_nouns = 0 if has_no_nouns else sum(
        (len(word) for word, pos, _, _ in candidate if pos == 'Noun'))

    scores = (num_nouns, num_words, has_no_nouns, len_sum_of_nouns)
    score = sum((score * weight for score, (_, weight) in zip(scores, my_weights)))
    return score

def oneDArray(x):
    return list(itertools.chain(*x))

def getTopics(model):
    topics = []
    for topic in model.print_topics(num_words=500):
        i=1
        model_words=[]
        topic_words=str(topic).split('"')
        for words in topic_words:
            if i%2==0:
                model_words.append(words)
            i+=1
        topics.append(model_words)
    return topics

def save_to_csv(output, keyword, type, tf_df):
    # save to csv    
    filename = "./output/" + output + "_" + type + "_" + keyword.replace(" ","") + ".csv"   
    # filename_list.append(filename)
    tf_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')


keywords, subkeywords, interested_words = interested_words()

In [3]:
# # keyword 전체 : 35개
# for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):

# ## 테스트 용 : 1개
keyword = keywords[0]
subkeyword = subkeywords[0]
interested_word = interested_words[0]

# 관심어(=명사) 추가
subkeyword = subkeyword.replace(" ", "").replace(",","|")
interested_word = subkeyword + "|" + interested_word.replace(" ", "").replace(",","|")

new_nouns = []
new_nouns = new_nouns + keyword.split(' ')
new_nouns.append((keyword.replace(" ", "")))
new_nouns = new_nouns + interested_word.split("|")
new_nouns = list(set(new_nouns))

# new_nouns
for nouns in new_nouns:
    twitter.add_dictionary(nouns, 'Noun')

In [4]:
df = hp.readall(keyword.replace(" ",""))
df = df[ (df['date'] >= '2019-07-01') & (df['date'] < '2020-07-01')]
df = df.drop_duplicates()

# df['text'] = df['title'].apply(hp.preprocessing) + df['content'].apply(hp.preprocessing)
df['text'] = df['title'] + df['content']
rows_date = df['date']

print(len(df['date']))

# sentences = df['text'].apply(hp.preprocessing)
            
# # 공백라인 및 NaN 제거
# while("" in sentences) : 
#     sentences.remove("") 
# while("NaN" in sentences) : 
#     sentences.remove("") 

# print(keyword)
# print(len(sentences))
# print(type(sentences))

2242


In [5]:
sentences = []
import re
for posts, dates in zip(df['text'], df['date']):
    for post in re.split('\?|\.|!', posts):
        post = hp.preprocessing(post)
        sentences.append(post)
            
# 공백라인 및 NaN 제거
while("" in sentences) : 
    sentences.remove("") 
while("NaN" in sentences) : 
    sentences.remove("") 

In [6]:
sentences

['연산동  외식1번가 점심회식 점심특선 1인 2만원 초반',
 '진갈비살, 양념갈비살, 그리고 물냉면까지점심 회식으로 연산동 외식1번가 다녀왔어요',
 '몇년전 다른 직장 다닐때 회식하러 온 이후로오랜만에 방문인데,점심특선은 원래 있었나요',
 '11시 30분부터 오후 3시까지1인 18,00025,000원으로 냉면 찌개 식사까지포함되어있다니 완전 괜찮더라고요',
 '외식1번가 자체가 다 규모가 큰데연제점 역시나 크고 좋았어요무엇보다 자연자연하게 조경해둔 인테리어가 고급스러움',
 '단체석도 많고 분위기도 좋아서어르신들도 단체로 많이 와서 식사 하시더라구요 양파절임, 맛있었지만더맛있는게 있었죠',
 '살얼음 끼얹은 백김치,',
 '이거 진짜진짜 너무 시원아삭 깔끔고기랑 꿀조합, 그리고 명이나물',
 '많이 먹고 왔어요  제 테이블은 5명이 앉아서 먹어서진갈비살 3인분 먼저 먹고양념갈비살',
 '2인분 이렇게  생갈비살 먼저 불위로 직 - 행 촤륵달궈진 불판 위에 고기 막 얹었을때 소리증말 좋아요 행복해지는 소리  소고기는 맛있어서 좋은데빨리 익으니까 더 좋아 함께나온 떡갈비도 부드럽고 달콤하다애들이 좋아할 맛',
 '버섯도 적당히 익었을때먹어주고',
 '열심히 고기를 굽고 자르고과하게 익지 않게 90%쯤 됐다싶으면가에로 빼줘요  오늘은 사진이 몇장 없네요 이게',
 '고기구우면서 찍느라 바빴어요 먹기도 먹어야하고  갈비살 먹고나서 양념갈비살',
 '확실히 양념이 양이 더 많아보여- 바짝 긴장한 상태로 굽기시작',
 '양념은 10에 9는 태우지 않나요',
 '소중한 고기를 태울수는 없지  생갈비살은 1-2번만 뒤집었다면양념갈비살은 수시로 뒤바주고 섞섞해줬어요,',
 '불판 한번 바꾸고나름 안태우고 잘 구버서 먹었어요 히히짜지않고 은은하게 달콤한 것이 냉면을 부른다  양념갈비살은 양이 꽤 넉넉하더라구요',
 '배는 찼었지만, 남은 3줄을 빠르게 구워주고냉면고기쌈 먹기 위해 킵해둡니다, 살얼음동동 물냉면',
 '여기 살얼음 맛집이야',
 '그릇들고 벌컥벌컥 마셔주니

In [7]:
# ###########################################################################################
from soynlp.word import WordExtractor
word_extractor = WordExtractor()
word_extractor.train(sentences)
word_scores = word_extractor.extract()

from soynlp.tokenizer import LTokenizer
cohesion_scores = {word:score.cohesion_forward for word, score in word_scores.items()}
ltokenizer = LTokenizer(scores = cohesion_scores)

training was done. used memory 0.433 Gbory 0.283 Gb
all cohesion probabilities was computed. # words = 29555
all branching entropies was computed # words = 44803
all accessor variety was computed # words = 44803


In [8]:
tokened_sentences = []
for sentence in sentences:
   tokened_sentences.append (ltokenizer.tokenize(sentence))

In [9]:
tokened_sentences

[['연산동', '외식', '1번가', '점심', '회식', '점심', '특선', '1인', '2만원', '초반'],
 ['진갈비살,',
  '양념갈비',
  '살,',
  '그리고',
  '물냉면',
  '까지점심',
  '회식으로',
  '연산동',
  '외식',
  '1번가',
  '다녀왔어요'],
 ['몇년',
  '전',
  '다른',
  '직장',
  '다닐때',
  '회식',
  '하러',
  '온',
  '이후로오랜만에',
  '방문',
  '인데,점심특선은',
  '원래',
  '있었',
  '나요'],
 ['11시',
  '30분',
  '부터',
  '오후',
  '3시까지',
  '1인',
  '18,000',
  '25,000원으로',
  '냉면',
  '찌개',
  '식사',
  '까지포함되어있다니',
  '완전',
  '괜찮',
  '더라고요'],
 ['외식',
  '1번가',
  '자체가',
  '다',
  '규모',
  '가',
  '큰데',
  '연제점',
  '역시',
  '나',
  '크고',
  '좋았어요',
  '무엇보다',
  '자연자연하게',
  '조경해둔',
  '인테리어',
  '가',
  '고급스러',
  '움'],
 ['단체석',
  '도',
  '많고',
  '분위기',
  '도',
  '좋아',
  '서어르신들도',
  '단체로',
  '많이',
  '와서',
  '식사',
  '하시더라구요',
  '양파절임,',
  '맛있',
  '었지만더맛있는게',
  '있었',
  '죠'],
 ['살얼음', '끼얹', '은', '백김치,'],
 ['이거', '진짜', '진짜', '너무', '시원', '아삭', '깔끔', '고기랑', '꿀조합', ',', '그리고', '명이나물'],
 ['많이',
  '먹고',
  '왔어요',
  '제',
  '테이',
  '블은',
  '5명이',
  '앉아서',
  '먹어',
  '서진갈비살',
  '3인분',
  '먼저',
  '먹고',
  '양념갈비살'],
 ['2인분',
  '

In [10]:
ts = []
for s in tokened_sentences:
    ts.append(hp.remove_stopwords(s))
#     print(s)
    

In [11]:
# # 공백라인 및 NaN 제거
while("" in s) : 
    ts.remove("") 
while("NaN" in sentences) : 
    ts.remove("") 

In [12]:
ts

[['연산동', '외식', '1번가', '점심', '회식', '점심', '특선', '1인', '2만원', '초반'],
 ['진갈비살,', '양념갈비', '살,', '물냉면', '까지점심', '회식으로', '연산동', '외식', '1번가', '다녀왔어요'],
 ['몇년',
  '직장',
  '다닐때',
  '회식',
  '하러',
  '이후로오랜만에',
  '방문',
  '인데,점심특선은',
  '원래',
  '있었',
  '나요'],
 ['30분',
  '3시까지',
  '1인',
  '25,000원으로',
  '냉면',
  '찌개',
  '식사',
  '까지포함되어있다니',
  '괜찮',
  '더라고요'],
 ['외식',
  '1번가',
  '자체가',
  '규모',
  '큰데',
  '연제점',
  '크고',
  '좋았어요',
  '무엇보다',
  '자연자연하게',
  '조경해둔',
  '인테리어',
  '고급스러'],
 ['단체석',
  '많고',
  '분위기',
  '서어르신들도',
  '단체로',
  '식사',
  '하시더라구요',
  '양파절임,',
  '맛있',
  '었지만더맛있는게',
  '있었'],
 ['살얼음', '끼얹', '백김치,'],
 ['진짜', '진짜', '시원', '아삭', '깔끔', '고기랑', '꿀조합', '명이나물'],
 ['왔어요', '테이', '블은', '5명이', '앉아서', '먹어', '서진갈비살', '3인분', '양념갈비살'],
 ['2인분',
  '생갈비',
  '불위로',
  '촤륵달궈진',
  '불판',
  '고기',
  '얹었을때',
  '소리증말',
  '행복',
  '해지는',
  '소리',
  '소고기',
  '맛있',
  '어서',
  '좋은',
  '데빨리',
  '익으',
  '니까',
  '떡갈비',
  '부드럽고',
  '달콤하',
  '다애들이'],
 ['버섯', '적당히', '익었을때먹어주고'],
 ['열심히',
  '고기',
  '굽고',
  '자르고과하게',
  '익지',
  '않게',
 

In [13]:
tsentences = []
for s in ts:
    line = ""
    for w in s:
        line = line + " " + w
    tsentences.append(line)

In [14]:
tsentences

[' 연산동 외식 1번가 점심 회식 점심 특선 1인 2만원 초반',
 ' 진갈비살, 양념갈비 살, 물냉면 까지점심 회식으로 연산동 외식 1번가 다녀왔어요',
 ' 몇년 직장 다닐때 회식 하러 이후로오랜만에 방문 인데,점심특선은 원래 있었 나요',
 ' 30분 3시까지 1인 25,000원으로 냉면 찌개 식사 까지포함되어있다니 괜찮 더라고요',
 ' 외식 1번가 자체가 규모 큰데 연제점 크고 좋았어요 무엇보다 자연자연하게 조경해둔 인테리어 고급스러',
 ' 단체석 많고 분위기 서어르신들도 단체로 식사 하시더라구요 양파절임, 맛있 었지만더맛있는게 있었',
 ' 살얼음 끼얹 백김치,',
 ' 진짜 진짜 시원 아삭 깔끔 고기랑 꿀조합 명이나물',
 ' 왔어요 테이 블은 5명이 앉아서 먹어 서진갈비살 3인분 양념갈비살',
 ' 2인분 생갈비 불위로 촤륵달궈진 불판 고기 얹었을때 소리증말 행복 해지는 소리 소고기 맛있 어서 좋은 데빨리 익으 니까 떡갈비 부드럽고 달콤하 다애들이',
 ' 버섯 적당히 익었을때먹어주고',
 ' 열심히 고기 굽고 자르고과하게 익지 않게 90% 됐다 싶으면가에로 빼줘요 사진 몇장 없네요',
 ' 고기 구우면서 찍느라 바빴어요 먹기도 먹어 야하고 갈비 나서 양념갈비',
 ' 양념 많아 보여- 바짝 긴장한 상태로 굽기 시작',
 ' 양념 9는 태우지 않나요',
 ' 소중한 고기 태울수는 없지 생갈비 살은 1-2번만 뒤집 었다면양념갈비살은 수시로 뒤바주고 섞섞해줬어요,',
 ' 불판 바꾸고 나름 안태우고 구버서 히히 짜지않고 은은하게 달콤한 냉면 부른다 양념갈비 살은 넉넉 하더라구요',
 ' 배는 찼었지만, 남은 3줄을 빠르게 구워 주고냉면고기쌈 킵해둡니다, 살얼음 동동 물냉면',
 ' 살얼음 맛집 이야',
 ' 그릇들고 벌컥벌컥 마셔주니고기먹은 기름이 깰끔하게슈르륵 내려가 기분 심심 깔끔한 맛이 어서저는 겨자 와사비 톡쏘는 좋아하 기때문에겨자 퐉퐉',
 ' 식초는 넣었어요 기다리 있었어',
 ' 냉면 갈비 쌈-',
 ' 보니까 먹고싶어 죽겠네요 평소

In [16]:
# from soynlp.noun import NewsNounExtractor
# noun_extractor = NewsNounExtractor()
# nouns = noun_extractor.train_extract(tsentences) # list of str like

# len(nouns.keys())

# len(nouns)
# nouns

used default noun predictor; Sejong corpus based logistic predictor
/opt/conda/lib/python3.8/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (148310, 75293, 67350)
predicting noun score was done                                        
before postprocessing 24882
_noun_scores_ 5031
checking hardrules ... done / 5031떡볶+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 3185
extracted 163 compounds from eojeolss ... 7000 / 7666

{'예정입니다': NewsNounScore(score=0, frequency=8, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '소개합니다': NewsNounScore(score=0, frequency=7, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '시작합니다': NewsNounScore(score=0, frequency=23, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '모습입니다': NewsNounScore(score=0, frequency=5, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '유리합니다': NewsNounScore(score=0, frequency=28, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '상황입니다': NewsNounScore(score=0, frequency=5, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '이탈리아': NewsNounScore(score=0, frequency=10, feature_proportion=0, eojeol_proportion=

In [ ]:
# list(nouns.values())[0]

In [ ]:
# list(nouns)[0]

In [21]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=1,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(sentences) # list of str or like
words = word_extractor.extract()

training was done. used memory 0.656 Gbory 0.634 Gb
all cohesion probabilities was computed. # words = 342514
all branching entropies was computed # words = 213852
all accessor variety was computed # words = 213852


In [22]:
words[keyword.replace(" ", "")]

Scores(cohesion_forward=0.058141241235083456, cohesion_backward=0.07857915169851111, left_branching_entropy=-0.0, right_branching_entropy=-0.0, left_accessor_variety=1, right_accessor_variety=1, leftside_frequency=1, rightside_frequency=1)

In [23]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

# print('단어   (빈도수, cohesion, branching entropy)\n')
nouns = []
freqs = []
cohesions = []
entropys = []
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:500]:
    nouns.append(word)
    freqs.append(score.leftside_frequency)
    cohesions.append(score.cohesion_forward)
    entropys.append(score.right_branching_entropy)

In [25]:
words_df = pd.DataFrame({"단어": nouns, "연관성": cohesions, "빈도수": freqs, "엔트로피": entropys})
words_df = words_df.sort_values(['빈도수','연관성'])

save_to_csv("3차", keyword, "연관성_빈도", words_df)

In [18]:
# 최빈어
tf_list = []
for sentence1 in nouns:
    sentence1.split()
    tf_list.append(sentence1.split())

lst = oneDArray(tf_list)
# lst

lst =  hp.remove_stopwords(lst)

In [19]:
lst

['예정입니다',
 '소개합니다',
 '시작합니다',
 '모습입니다',
 '유리합니다',
 '상황입니다',
 '이탈리아',
 '중화요리',
 '가격대비',
 '중요합니다',
 '의미합니다',
 '해당합니다',
 '제육볶음',
 '가족끼리',
 '별내맛집',
 '고민고민',
 '가족들끼리',
 '이곳저곳',
 '지하주차장',
 '대표메뉴',
 '이것저것',
 '오후3시',
 '소고기무한리필',
 '감사합니다',
 '전략연구소',
 '오랫동안',
 '생선튀김',
 '고객님들',
 '집게다리',
 '주차가능',
 '사용한다고',
 '초딩입맛',
 '사랑입니다',
 '간장소스',
 '한우고기',
 '위생등급',
 '마늘양념',
 '전통시장',
 '미세먼지',
 '생선조림',
 '한우육회',
 '육회비빔밥',
 '좌식테이블',
 '11시30분',
 '장어튀김',
 '부대찌개',
 '마늘보쌈',
 '두부튀김',
 '조리과정',
 '표고버섯',
 '오픈한지',
 '수산시장',
 '메인요리',
 '주말외식',
 '비닐장갑',
 '업종변경창업',
 '감자튀김',
 '보쌈배달',
 '감자샐러드',
 '단품메뉴',
 '고추장불고기',
 '해물뚝배기',
 '갈치조림',
 '제공한다고',
 '가득가득',
 '사용가능',
 '매콤달콤',
 '청양고추',
 '해초비빔밥',
 '잔치국수',
 '일상밥상',
 '사진찍기',
 '오늘저녁',
 '어마무시',
 '제공합니다',
 '해물칼국수',
 '중국당면',
 '동탄점심맛집',
 '감동식당',
 '제철밥상',
 '초록초록',
 '한방육수',
 '인기메뉴',
 '요리전문점',
 '지원사업',
 '제주도민',
 '연어샐러드',
 '전년대비',
 '전문점입니다',
 '양념불고기',
 '조리방법',
 '대전맛집',
 '정보기술',
 '수유본점',
 '추석연휴',
 '크림소스',
 '연말모임장소',
 '키친박스',
 '한달동안',
 '해동밥상',
 '아기자기',
 '한상가득',
 '포장·배달',
 '자랑합니다',
 '남양주맛집',
 '이동갈비',
 '

In [20]:
# 선별된 품사별 빈도수 계산 & 상위 빈도 xxx위 까지 출력
counts = Counter(lst)

toplist = counts.most_common(500)
# type(toplist)

toplist

[('예정입니다', 1),
 ('소개합니다', 1),
 ('시작합니다', 1),
 ('모습입니다', 1),
 ('유리합니다', 1),
 ('상황입니다', 1),
 ('이탈리아', 1),
 ('중화요리', 1),
 ('가격대비', 1),
 ('중요합니다', 1),
 ('의미합니다', 1),
 ('해당합니다', 1),
 ('제육볶음', 1),
 ('가족끼리', 1),
 ('별내맛집', 1),
 ('고민고민', 1),
 ('가족들끼리', 1),
 ('이곳저곳', 1),
 ('지하주차장', 1),
 ('대표메뉴', 1),
 ('이것저것', 1),
 ('오후3시', 1),
 ('소고기무한리필', 1),
 ('감사합니다', 1),
 ('전략연구소', 1),
 ('오랫동안', 1),
 ('생선튀김', 1),
 ('고객님들', 1),
 ('집게다리', 1),
 ('주차가능', 1),
 ('사용한다고', 1),
 ('초딩입맛', 1),
 ('사랑입니다', 1),
 ('간장소스', 1),
 ('한우고기', 1),
 ('위생등급', 1),
 ('마늘양념', 1),
 ('전통시장', 1),
 ('미세먼지', 1),
 ('생선조림', 1),
 ('한우육회', 1),
 ('육회비빔밥', 1),
 ('좌식테이블', 1),
 ('11시30분', 1),
 ('장어튀김', 1),
 ('부대찌개', 1),
 ('마늘보쌈', 1),
 ('두부튀김', 1),
 ('조리과정', 1),
 ('표고버섯', 1),
 ('오픈한지', 1),
 ('수산시장', 1),
 ('메인요리', 1),
 ('주말외식', 1),
 ('비닐장갑', 1),
 ('업종변경창업', 1),
 ('감자튀김', 1),
 ('보쌈배달', 1),
 ('감자샐러드', 1),
 ('단품메뉴', 1),
 ('고추장불고기', 1),
 ('해물뚝배기', 1),
 ('갈치조림', 1),
 ('제공한다고', 1),
 ('가득가득', 1),
 ('사용가능', 1),
 ('매콤달콤', 1),
 ('청양고추', 1),
 ('해초비빔밥', 1),
 ('잔

In [ ]:
toplist = counts.most_common(500)
# type(toplist)

tf_df = pd.DataFrame (toplist,columns=['단어', '빈도수'])
# tf_df
# save_to_csv(keyword, "최빈어", tf_df)
save_to_csv("3차", keyword, "최빈어", tf_df)

In [ ]:
###########################################################################################

# 토픽 모델링
posts = words
dataset = pd.DataFrame(posts, columns=['문장'])

#     dataset.head()

# nan_value = float("NaN")
# dataset.replace("", nan_value, inplace=True)
# dataset.dropna(subset = ["문장"], inplace=True)
# dataset.reindex



dataset['문장']

tmp_corpus = dataset['문장'].map(lambda x: x.split('.'))


for i, line in enumerate(dataset['문장']):
#     print(type(line))
    s = remove_stopwords(line.split(" "))
#     print( ' '.join([str(elem) for elem in s] ))
#     print(s)
    dataset['문장'][i] = ' '.join([str(elem) for elem in s]) 
#     tmp_corpus

#     type(tmp_corpus)
#     tmp_corpus[11]
#     tmp_corpus[13]

# corpus [[w1,w2,w3..],[..]]
corpus = []
for i in range(len(tmp_corpus)):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)
#   corpus

num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

In [ ]:


nouns = corpus
bigram = gensim.models.Phrases(nouns)
trigram = gensim.models.Phrases(bigram[nouns])
bigram_model = gensim.models.phrases.Phraser(bigram)
trigram_model = gensim.models.phrases.Phraser(trigram)

bigram_document = [bigram_model[nouns] for nouns in nouns]
# bigram_document[1]
# bigram_document[3]

id2word = corpora.Dictionary(bigram_document)
corpus = [id2word.doc2bow(doc) for doc in bigram_document]
# corpus[0]

# corpus[7]  # 단어별 출현 출현 수

### Topic Coherence 계산

# 토픽 모델링을 수행함에 있어서 적절한 토픽의 갯수를 찾는 것이 중요하다. 
# 2부터 9까지 값을 늘려가면서 LDA 모델을 생성하여 각 모델의 coherence를 계산한다.
# 그리고, 적절한 토픽의 수는 토픽 갯수를 늘려가며 높은 coherence score 를 가지는 값으로 결정한다

coherence_score=[]
for i in range(2,10):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=i)
    coherence_model = CoherenceModel(model, texts=bigram_document, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model.get_coherence()
    print('n=',i,'\nCoherence Score: ', coherence_lda)
    coherence_score.append(coherence_lda)

#     k=[]
#     for i in range(2,10):
#         k.append(i)

#     x=np.array(k)
#     y=np.array(coherence_score)
#     title = f'{keyword} Topic Coherence'
#     plt.title(title)
#     plt.plot(x,y)
#     plt.xlim(2,10)
#     plt.xlabel('Number Of Topic (2-10)')
#     plt.ylabel('Cohrence Score')
#     plt.show()

model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=3)
# model.print_topics()
model.print_topics(num_words=500) # num_topics=20

_topics = getTopics(model)
#_topics

#type(_topics)

co_top_df = pd.DataFrame(_topics)

co_top_df.set_index(0)

save_to_csv("3차", keyword, "토픽", co_top_df)

In [ ]:
# len(lst)

# nouns_df = pd.DataFrame (lst,columns=['단어'])

# nouns_df.head()

# nouns = nouns_df['단어'].apply(hp.preprocessing)

# len(nouns)

In [ ]:
# df.head()
# dataset = df[['text','date']]
# dataset

# posts = words
# dataset = pd.DataFrame(posts, columns=['문장'])

#     dataset.head()

# nan_value = float("NaN")
# dataset.replace("", nan_value, inplace=True)
# dataset.dropna(subset = ["text"], inplace=True)
# dataset.reindex

# sentences = []
# import re
# for posts, dates in zip(dataset['text'], dataset['date']):
#     for post in re.split(', |\?|~|,+|\.|_|-|!|\+', posts):
#         post = re.sub('\s+|\t|\n', ' ', post).strip()
#         post = re.sub('ㅎ+|ㅋ+|ㅠ+|\^+|ㅜ+|▶+|♥+|ㄱ+', '', post)
#         post = re.sub('^[0-9].*[0-9]$', '', post)
#         post = re.sub('^[0-9]+일$', '', post)
#         post = re.sub('^[0-9]+시$', '', post)
#         post = re.sub('.*(광고정보).*', '', post)
#         post = re.sub('.*(기사출처).*', '', post)
#         post = re.sub('[\'\"]', '', post)
#         post = re.sub('[\'\n"]', '', post)
#         post = re.sub('^[http|naver|com|daum].*', '', post)
#         post = re.sub('/|『|』|‘|\[|\]', ' ', post)

#         sentences.append(post)
# sentences = df['text'].apply(hp.preprocessing)
            
# # 7 공백라인 및 NaN 제거
# while("" in sentences) : 
#     sentences.remove("") 
# while("NaN" in sentences) : 
#     sentences.remove("") 

# sentences

In [ ]:
# df.head()
# dataset = df[['text','date']]
# dataset

# posts = words
# dataset = pd.DataFrame(posts, columns=['문장'])

#     dataset.head()

# nan_value = float("NaN")
# dataset.replace("", nan_value, inplace=True)
# dataset.dropna(subset = ["text"], inplace=True)
# dataset.reindex


# from ckonlpy.tag import Postprocessor  # https://github.com/lovit/customized_konlpy

# sentences_tag = []
# for sentence in sentences:
# #     passwords = {'아이오아이', ('정말', 'Noun')}
# #     postprocessor = Postprocessor(twitter, passwords = passwords)
#     postprocessor = Postprocessor(twitter, stopwords = sw.stopwords_kr)
#     morph = postprocessor.pos(sentence) # morph = twitter.pos(sentence)
#     sentences_tag.append(morph)

# sentences_tag

# dataset['text']
# tmp_corpus = dataset['text'].map(lambda x: x.split('.'))
# tmp_corpus.describe()

# posts = []
# for sentence1 in sentences_tag:
#     words = ""
#     for word, tag in sentence1:
# #         print(word)
#         if tag in ['Noun']:     # 명사만 추출, # 명사/형용사 추출은 ['Noun','Adjective']
#             words = words + word + " "
#     posts.append(words)

# print(len(posts), type(posts))
# print(len(rows_date), type(rows_date))

# rows

# posts

# # 후처리

# while("" in posts) : 
#     posts.remove("") 
# while("NaN" in posts) : 
#     posts.remove("")

# posts